In [3]:
import os
import cv2
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter

In [4]:
def load_data(base_dir, img_size, splits):
    if isinstance(splits, str):
        splits = [splits]

    data, labels = [], []
    category = "stenosis"

    for split in splits:
        img_dir = os.path.join(base_dir, category, split, "images")
        ann_file = os.path.join(base_dir, category, split, "annotations", f"{split}.json")

        if not os.path.exists(ann_file):
            print(f"Missing annotation file: {ann_file}")
            continue

        with open(ann_file, "r") as f:
            annotations = json.load(f)

        for img_info in annotations.get("images", []):
            img_path = os.path.join(img_dir, img_info["file_name"])
            stenosis_severity = img_info.get("stenosis_percentage", 0)

            if 10 < stenosis_severity < 50:
                label = 0
            elif 50 <= stenosis_severity < 70:
                label = 1
            else:
                label = 2

            if os.path.exists(img_path):
                img = cv2.imread(img_path)
                img = cv2.resize(img, img_size)
                img = img / 255.0

                data.append(img)
                labels.append(label)

    return np.array(data, dtype=np.float32), np.array(labels)


In [3]:
x_train, y_train = load_data(".", (224, 224), splits="train")


In [4]:
y_train=to_categorical(y_train, num_classes=3)

In [5]:
x_val, y_val = load_data(".", (224, 224), splits="val")


In [6]:
y_val=to_categorical(y_val, num_classes=3)

In [7]:
print(x_train.shape)
print(y_train.shape)

(1000, 224, 224, 3)
(1000, 3)


In [8]:
print(x_val.shape)
print(y_val.shape)

(200, 224, 224, 3)
(200, 3)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

In [10]:
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    
    Conv2D(64, (3, 3), activation="relu", kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    
    Conv2D(128, (3, 3), activation="relu", kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    
    Conv2D(256, (3, 3), activation="relu", kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Flatten(),
    Dense(256, activation="relu", kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    
    Dense(3, activation="softmax") 
])

c:\Users\Sriram\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     9,437,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,828,547 (37.49 MB)

 Trainable params: 9,827,587 (37.49 MB)

 Non-trainable params: 960 (3.75 KB)

In [12]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [13]:
callbacks_list = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=4,                 
        restore_best_weights=True,
        verbose=1
    ),

    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.4,
        patience=2,
        min_lr=1e-7,
        verbose=1
    )
]

In [15]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=callbacks_list
)

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 845ms/step - accuracy: 1.0000 - loss: 0.4672 - val_accuracy: 0.9850 - val_loss: 0.8764 - learning_rate: 0.0010
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 846ms/step - accuracy: 1.0000 - loss: 0.4317 - val_accuracy: 0.9750 - val_loss: 0.9921 - learning_rate: 0.0010
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 802ms/step - accuracy: 1.0000 - loss: 0.3996
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0004000000189989805.
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 837ms/step - accuracy: 1.0000 - loss: 0.3994 - val_accuracy: 0.9650 - val_loss: 1.2011 - learning_rate: 0.0010
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 841ms/step - accuracy: 1.0000 - loss: 0.3741 - val_accuracy: 0.9750 - val_loss: 1.0986 - learning_rate: 4.0000e-04
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 839ms/step - accuracy: 1.0000 - loss: 0.3629
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00016000000759959222.
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 874ms/step - accuracy: 1.0000 - loss: 0

In [5]:
x_test,y_test=load_data(".", (224, 224), splits="test")

In [6]:
y_test=to_categorical(y_test,num_classes=3)

In [18]:
print(x_test.shape)
print(x_test.shape)

(300, 224, 224, 3)
(300, 224, 224, 3)


In [19]:
from sklearn.metrics import accuracy_score

In [ ]:
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Test Accuracy: {accuracy:.2f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 172ms/step
Test Accuracy: 0.98


In [21]:
model.save('angiogram.h5')